In [1]:
import requests
import pandas as pd
import json
import getpass
rapidAPIkey = getpass.getpass()

········


## Defining the function

For this function I chose to compare flights from 'Schiphol' and 'New York John F. Kennedy'. 

In [2]:
def airport_code(airport):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"
    params = {"query":airport}
    
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
           'x-rapidapi-key': rapidAPIkey}
    
    response = requests.get(url, headers = headers, params = params)
    
    return response.json()["Places"][0]["PlaceId"]

airport_code('Schiphol')

'AMS-sky'

In [3]:
def flight_prices(departure, arrival, date):

    departure_code = airport_code(departure)
    arrival_code = airport_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': rapidAPIkey}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

flight_prices('Schiphol', 'New York John F. Kennedy', '2021-10-10')

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 330,
   'Direct': False,
   'OutboundLeg': {'CarrierIds': [1218],
    'OriginId': 40595,
    'DestinationId': 60987,
    'DepartureDate': '2021-10-10T00:00:00'},
   'QuoteDateTime': '2021-09-14T14:28:00'},
  {'QuoteId': 2,
   'MinPrice': 367,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [1324],
    'OriginId': 40595,
    'DestinationId': 60987,
    'DepartureDate': '2021-10-10T00:00:00'},
   'QuoteDateTime': '2021-09-14T14:28:00'}],
 'Carriers': [{'CarrierId': 1324, 'Name': 'KLM'},
  {'CarrierId': 1218, 'Name': 'Iberia'}],
 'Places': [{'Name': 'Amsterdam',
   'Type': 'Station',
   'PlaceId': 40595,
   'IataCode': 'AMS',
   'SkyscannerCode': 'AMS',
   'CityName': 'Amsterdam',
   'CityId': 'AMST',
   'CountryName': 'Netherlands'},
  {'Name': 'New York John F. Kennedy',
   'Type': 'Station',
   'PlaceId': 60987,
   'IataCode': 'JFK',
   'SkyscannerCode': 'JFK',
   'CityName': 'New York',
   'CityId': 'NYCA',
   'CountryName': 'United States

In [4]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

dates = get_dates("2021-10-10", "2021-10-11")

In [5]:
def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    return {date:flight_prices(origin, destination, date) for date in dates}

json = flight_days('Schiphol', 'New York John F. Kennedy', '2021-10-09', '2021-10-10')

In [6]:
pd.json_normalize(json)

,2021-10-09.Quotes,2021-10-09.Carriers,2021-10-09.Places,2021-10-09.Currencies,2021-10-10.Quotes,2021-10-10.Carriers,2021-10-10.Places,2021-10-10.Currencies
0,"[{'QuoteId': 1, 'MinPrice': 265, 'Direct': Fal...","[{'CarrierId': 1324, 'Name': 'KLM'}, {'Carrier...","[{'Name': 'Amsterdam', 'Type': 'Station', 'Pla...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...","[{'QuoteId': 1, 'MinPrice': 330, 'Direct': Fal...","[{'CarrierId': 1324, 'Name': 'KLM'}, {'Carrier...","[{'Name': 'Amsterdam', 'Type': 'Station', 'Pla...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."


### Dataframe for 2021-10-10

In [7]:
quotes_10 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-10.Quotes"][0])
quotes_10 = quotes_10['MinPrice']

In [8]:
airport_10 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-10.Places"][0])
airport_10 = airport_10['Name']

In [9]:
carriers_10 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-10.Carriers"][0])
carriers_10 = carriers_10.rename(columns={'Name':'Carrier'})
carriers_10 = carriers_10['Carrier']

In [10]:
flights_10 = pd.concat([quotes_10,airport_10,carriers_10],axis=1)

In [11]:
flights_10['date'] = '2021-10-10'

### Dataframe for 2021-10-09

In [12]:
quotes_09 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-09.Quotes"][0])
quotes_09 = quotes_09['MinPrice']

In [13]:
airport_09 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-09.Places"][0])
airport_09 = airport_09['Name']

In [14]:
carriers_09 = pd.DataFrame(pd.DataFrame(pd.json_normalize(json))["2021-10-09.Carriers"][0])
carriers_09 = carriers_09.rename(columns={'Name':'Carrier'})
carriers_09 = carriers_09['Carrier']

In [15]:
flights_09 = pd.concat([quotes_09,airport_09,carriers_09],axis=1)

In [16]:
flights_09['date'] = '2021-10-09'

## Final dataframe

In [17]:
flights = pd.concat([flights_09,flights_10],axis=0)

In [18]:
flights = flights[flights['Name']== 'Amsterdam']
flights['Arrival'] = 'New York John F. Kennedy'
flights = flights.rename(columns={'Name':'Departure'})

,MinPrice,Departure,Carrier,date,Arrival
0,265,Amsterdam,KLM,2021-10-09,New York John F. Kennedy
0,330,Amsterdam,KLM,2021-10-10,New York John F. Kennedy


In [19]:
flights = flights[['MinPrice','Departure','Arrival','Carrier','date']]
flights

,MinPrice,Departure,Arrival,Carrier,date
0,265,Amsterdam,New York John F. Kennedy,KLM,2021-10-09
0,330,Amsterdam,New York John F. Kennedy,KLM,2021-10-10
